# Exercise 1.1
## This *.ipynb file contains:

- `LQRsolver`: a class required to be written,which can be __initialised__ with default time horizon T = 1 (which can also take value from users' input) and with the matrices specifying the LQR problem which are:
    - `H`(a `torch.Size([n,n])` `torch.tensor`): the linear relations of dynamics between the total `n` state processes.
    - `M`(a `torch.Size([n,m])` `torch.tensor`): the influences from `m` control variables to `n` state processes. 
    - `sigma`(a `torch.Size([n,d])` `torch.tensor`): the diffusion matrix from `d` Wiener processes to `n` state processes.
    - `C`(a `torch.Size([n,n])` `torch.tensor`): the contribution matrix from state processes to runnning reward.
    - `D`(a `torch.Size([m,m])` `torch.tensor`): the contribution matrix from the final value of state processes to runnning reward.
    - `R`(a `torch.Size([n,n])` `torch.tensor`): the contribution matrix from the final values of state processes to terminal reward.
    
    __Declaration for dimensions of matrices__:
    Here `n` should be compatible with the dimension of state variable space and m is the dimension of control variable space.
    Note that `n = m = 2` and `d = 1` in this exercise but can be extended to a higher dimension.

    There are __3 main methods__ built in `LQRsolver` including

    - `solve_riccati_ode`: __a numerical solver for Ricatti ODE__ which requires
    
        __input__ of
    
        - `time_grid` (a `torch.Size([l]) torch.Tensor`)
        
        and __returns__
        
        - the values of solution function $S(t)$ (a `torch.Size([l,n,n])` `torch.tensor`)
        
        (Two numerical methods are tried to be provided as options: Euler scheme and 4th order Runge-Kutta scheme)
    
    - `value_function`: __a computation of value function__ which requires
        __inputs__ (in sequence) of 
        
        - `t_batch`(a `torch.Size([batch_size])` `torch.tensor` whose entries took value initially from [0,1] but would be scaled by the given T in the further calculation)
        - `x_batch`(a `torch.Size([batch_size,1,n])` `torch.tensor`)
        
        and __returns__
        
        - values of value_function (a `torch.Size([batch_size])` `torch.tensor`)
        
        
    - `markov_control`: __a computation of Markov control function__ which requires
        __inputs__ (in sequence) of 
        
        - `t_batch`(a `torch.Size([batch_size])` `torch.tensor` whose entries took value initially from [0,1] but would be scaled by the given T in the further calculation)
        - `x_batch`(a `torch.Size([batch_size,1,n])` `torch.tensor`)
        
        and __returns__
        
        - values of Markov_control_function (a `torch.Size([batch_size,n])` `torch.tensor`)
        
        
        
- __A runnable sample__ inclueding: 
    - a whole set of matrices for __initialisation__
    - an example of __calculation of value function__ with given t_batch and x_batch
    - an example of __calculation of Markov control function__ with given t_batch and x_batch


In [49]:
import torch
import numpy as np
from scipy.interpolate import CubicSpline
from concurrent.futures import ThreadPoolExecutor

import warnings #for alarming some inappropriate inputs

class LQRSolver:
    #default numerical method is Euler.
    #Euler and Runge-Kutta are supported in this code.
    def __init__(self, H, M, sigma, C, D, R, T = 1, method="euler"):
        
        if not self.is_semi_positive_definite(C):
            raise ValueError("Matrix C must be semi-positive definite.")
        if not self.is_semi_positive_definite(R):
            raise ValueError("Matrix R must be semi-positive definite.")

        if not self.is_positive_definite(D):
            raise ValueError("Matrix D must be positive definite.")
 

        self.H = H
        self.M = M
        self.sigma = sigma
        self.C = C
        self.D = D
        self.R = R
        self.T = T
        self.method = method 
    
    def is_positive_definite(self, matrix):
        
        eigvals, _ = torch.linalg.eig(matrix)
        real_parts = eigvals.real
        return torch.all(real_parts > 0) 

    def is_semi_positive_definite(self, matrix):

        eigvals, _ = torch.linalg.eig(matrix)
        real_parts = eigvals.real
        return torch.all(real_parts >= 0) 

    def solve_riccati_ode(self, time_grid):
        
        if not isinstance(time_grid, torch.Tensor):
            raise TypeError("time_grid should be an 1-D torch.Tensor")
        else:
            if not ((np.abs(time_grid[-1] - self.T) <= 1e-12) or (time_grid[0]>=0)):
                raise Exception("Please ensure that the first entry of time_grid is 0 and the last entry is equal to T.")
            else:
                time_grid_in = time_grid

                    
        time_grid_in = torch.flip(time_grid_in, dims=[0])
        S = self.R.clone()
        S_list = []
        S_list.append(S)

        for i in range(1, len(time_grid_in)):
            dt = time_grid_in[i] - time_grid_in[i-1]
            #Notice that here every dt is a negative value for backward scheme.
            if self.method == "euler":
                S = self.euler_step(S, dt)
            elif self.method == "rk4":
                S = self.rk4_step(S, dt)
            else:
                raise ValueError("Unsupported method")
            S_list.append(S)
        
        return torch.stack(S_list[::-1])

    def euler_step(self, S, dt):
        dS = -2 * self.H.T @ S + S @ self.M @ torch.inverse(self.D) @ self.M.T @ S - self.C
        return S + dS * dt

    def rk4_step(self, S, dt):
        def riccati_derivative(S):
            return -2 * self.H.T @ S + S @ self.M @ torch.inverse(self.D) @ self.M.T @ S - self.C
        k1 = riccati_derivative(S)
        k2 = riccati_derivative(S + 0.5 * dt * k1)
        k3 = riccati_derivative(S + 0.5 * dt * k2)
        k4 = riccati_derivative(S + dt * k3)

        return S + (k1 + 2*k2 + 2*k3 + k4) * dt / 6

    def value_function(self, t_batch, x_batch):

        # Verify the shapes of the inputs.

        if not (t_batch.dim() == 1 and torch.all((t_batch >= 0) & (t_batch <= 1))):
            raise TypeError("t_batch should be a 1D tensor in which every entry is in [0,1].")
        else:
            if not (x_batch.dim() == 3 and x_batch.size()[0] == len(t_batch) and x_batch.size()[1] == 1 and x_batch.size()[2] == self.H.size()[0]):
                raise TypeError("x_batch should have shape (%d, 1, %d)."%(len(t_batch),self.H.size(2)))
        
        time_grids = [torch.linspace(float(t), self.T, 5000, dtype=torch.float32) for t in t_batch]
        
        with ThreadPoolExecutor(max_workers=len(time_grids)) as executor:
            S_tensor_list = list(executor.map(self.solve_riccati_ode, time_grids))
        
        S_tensor_tensor = torch.stack(S_tensor_list) 
        print(S_tensor_tensor.shape)
        S_t_s = S_tensor_tensor[:, 0, :, :]
        
        x = x_batch.squeeze(1)
        
        xTSx = torch.einsum('bi, bij, bj -> b', x, S_t_s , x)
        
        time_grids_tensor = torch.stack(time_grids)

        dts = time_grids_tensor[:, 1:] - time_grids_tensor[:, :-1]
                    
        sigma_sigma_T = self.sigma @ self.sigma.T

        traces =torch.diagonal((sigma_sigma_T @ S_tensor_tensor[:, :-1]), dim1=-2, dim2=-1).sum(dim=-1)
  
        integral_part = torch.sum(traces * dts, dim=1)

        v_tx = xTSx + integral_part
        
        return v_tx

    def markov_control(self, t_batch, x_batch):
        
        # Verify the shapes of the inputs.

        if not (t_batch.dim() == 1 and torch.all((t_batch >= 0) & (t_batch <= 1))):
            raise TypeError("t_batch should be a 1D tensor in which every entry is in [0,1].")
        else:
            if not (x_batch.dim() == 3 and x_batch.size()[0] == len(t_batch) and x_batch.size()[1] == 1 and x_batch.size()[2] == self.H.size()[0]):
                raise TypeError("x_batch should have shape (%d, 1, %d)."%(len(t_batch),self.H.size(2)))
        
        time_grids = [torch.linspace(float(t), self.T, 5000, dtype=torch.float32) for t in t_batch]
        
        with ThreadPoolExecutor(max_workers=len(time_grids)) as executor:
            S_tensor_list = list(executor.map(self.solve_riccati_ode, time_grids))
        
        S_tensor_tensor = torch.stack(S_tensor_list) 
        
        S_t_s = S_tensor_tensor[:, 0, :, :]

        MT = self.M.T
        D_inv = torch.inverse(self.D)
        x = torch.transpose(x_batch,dim0 = 2,dim1 = 1)
        a_tx = -D_inv @ MT @ S_t_s @x
        a_tx = torch.transpose(a_tx,dim0 = 1,dim1 = 2).squeeze()

        return a_tx

In [51]:
#Initialization
H = torch.tensor([[1.0, 2.0], [-2.0, -3.0]], dtype=torch.float32)
M = torch.tensor([[1.0,0.0], [0.0,1.0]], dtype=torch.float32)
sigma = torch.tensor([[0.5], [0.5]], dtype=torch.float32)  
C = torch.tensor([[2.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive semi-definite
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive definite
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]], dtype=torch.float32)  # Positive semi-definite
T = 5.0

In [53]:
solver = LQRSolver(H, M, sigma, C, D, R, T)

In [55]:
t_batch = torch.tensor(np.linspace(0.1,0.9,20))
x_batch = torch.rand([20,1,2])

In [57]:
solver.value_function(t_batch, x_batch)

torch.Size([20, 5000, 2, 2])


tensor([2.5316, 2.1126, 2.1881, 3.1743, 2.0243, 2.0091, 3.2592, 2.7073, 2.5760,
        2.9174, 3.1114, 2.1092, 2.4862, 2.1161, 1.9975, 2.2175, 2.9050, 1.7916,
        1.9471, 2.2321])

In [58]:
solver.markov_control(t_batch, x_batch)

tensor([[-0.6910, -0.4035],
        [-0.0847, -0.0891],
        [-0.2740, -0.1949],
        [-1.1415, -0.6578],
        [ 0.2075, -0.0274],
        [ 0.2156, -0.0329],
        [-1.3267, -0.7275],
        [-0.9111, -0.5285],
        [-0.6091, -0.4314],
        [-1.1239, -0.6288],
        [-1.1049, -0.6590],
        [-0.0581, -0.1971],
        [-0.6847, -0.4448],
        [-0.4935, -0.3006],
        [-0.2109, -0.1959],
        [-0.5629, -0.3609],
        [-0.7381, -0.5567],
        [ 0.0994, -0.0258],
        [-0.4669, -0.2647],
        [-0.6868, -0.4146]])